In [1]:
!nvidia-smi

Sun Jul 23 01:48:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!git clone https://github.com/HusseinMansourMohd/Enhancing-MedViT-Incorporating-Adapter-Modules-for-Improved-Medical-Image-Classification


Cloning into 'Enhancing-MedViT-Incorporating-Adapter-Modules-for-Improved-Medical-Image-Classification'...
remote: Enumerating objects: 324, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 324 (delta 9), reused 15 (delta 6), pack-reused 306
Receiving objects: 100% (324/324), 101.80 KiB | 4.24 MiB/s, done.
Resolving deltas: 100% (178/178), done.


In [3]:
import sys
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision.utils
import torchvision.datasets as dsets
from torchsummary import summary

In [4]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.1 MB/s eta 0:00:00


In [5]:
!pip install einops==0.6.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.1 MB/s eta 0:00:00


In [6]:
cd /content/Enhancing-MedViT-Incorporating-Adapter-Modules-for-Improved-Medical-Image-Classification/


/content/Enhancing-MedViT-Incorporating-Adapter-Modules-for-Improved-Medical-Image-Classification


In [7]:
!git clone https://github.com/HusseinMansourMohd/Deformable-DETR.git


Cloning into 'Deformable-DETR'...
remote: Enumerating objects: 124, done.
remote: Counting objects: 100% (124/124), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 124 (delta 42), reused 84 (delta 36), pack-reused 0
Receiving objects: 100% (124/124), 5.67 MiB | 17.22 MiB/s, done.
Resolving deltas: 100% (42/42), done.


In [8]:
%cd Deformable-DETR

/content/Enhancing-MedViT-Incorporating-Adapter-Modules-for-Improved-Medical-Image-Classification/Deformable-DETR


In [9]:
!cd /content/Enhancing-MedViT-Incorporating-Adapter-Modules-for-Improved-Medical-Image-Classification/Deformable-DETR/models/ops && python setup.py install

running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based t

In [10]:
%cd /content/Enhancing-MedViT-Incorporating-Adapter-Modules-for-Improved-Medical-Image-Classification/

/content/Enhancing-MedViT-Incorporating-Adapter-Modules-for-Improved-Medical-Image-Classification


In [11]:
import MedVit
print(dir(MedVit))

['Adapter', 'ConvBNReLU', 'DropPath', 'ECALayer', 'ECB', 'E_MHSA', 'F', 'LTB', 'LocalityFeedForward', 'MHCA', 'MedViT', 'MedViT_base', 'MedViT_large', 'MedViT_small', 'Mlp', 'NORM_EPS', 'PatchEmbed', 'SELayer', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_make_divisible', 'checkpoint', 'h_sigmoid', 'h_swish', 'math', 'merge_pre_bn', 'nn', 'normal_', 'partial', 'rearrange', 'register_model', 'torch', 'trunc_normal_']


In [12]:
!pip install medmnist

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116932 sha256=9f1fd24d0c2a7421667f423564ffa8542ad449355566373457efb75ce88b8059
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire


In [13]:
import medmnist
from medmnist import INFO
data_flag = 'retinamnist'
# [tissuemnist , pathmnist, chestmnist, dermamnist, octmnisr]
# ,pnemonismnist , retinamnist, bloodmnist, tissuemnist, organcmist, organs ]
download = True

NUM_EPOCHS = 15
BATCH_SIZE = 15
LR = 0.005

info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

DataClass = getattr(medmnist, info['python_class'])

In [14]:
import torchvision.transforms as transforms
#preprocessing
train_transform = transforms.Compose([
        transforms.Resize(224),
        transforms.Lambda(lambda image:image.convert('RGB')),
        transforms.AugMix(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])
]
)
test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.Lambda(lambda image:image.convert('RGB')),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
]
)

In [15]:
import torch.utils.data as data
#load the data
train_dataset = DataClass(split='train', transform=train_transform, download=download)
val_dataset = DataClass(split='val', transform=train_transform, download=download)
test_dataset = DataClass(split='test', transform=test_transform, download=download)

#encapsulate data into dataloader form
train_loader = data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
train_loader_at_eval = data.DataLoader(dataset=val_dataset, batch_size=2*BATCH_SIZE, shuffle=False)
test_loader = data.DataLoader(dataset=test_dataset, batch_size=2*BATCH_SIZE, shuffle=False)

100%|██████████| 3291041/3291041 [00:00<00:00, 26294575.08it/s]

Using downloaded and verified file: /root/.medmnist/retinamnist.npz
Using downloaded and verified file: /root/.medmnist/retinamnist.npz


In [16]:
print(train_dataset)
print("+++++++++++++++++")
print(test_dataset)

Dataset RetinaMNIST (retinamnist)
    Number of datapoints: 1080
    Root location: /root/.medmnist
    Split: train
    Task: ordinal-regression
    Number of channels: 3
    Meaning of labels: {'0': '0', '1': '1', '2': '2', '3': '3', '4': '4'}
    Number of samples: {'train': 1080, 'val': 120, 'test': 400}
    Description: The RetinaMNIST is based on the DeepDRiD challenge, which provides a dataset of 1,600 retina fundus images. The task is ordinal regression for 5-level grading of diabetic retinopathy severity. We split the source training set with a ratio of 9:1 into training and validation set, and use the source validation set as the test set. The source images of 3×1,736×1,824 are center-cropped and resized into 3×28×28.
    License: CC BY 4.0
+++++++++++++++++
Dataset RetinaMNIST (retinamnist)
    Number of datapoints: 400
    Root location: /root/.medmnist
    Split: test
    Task: ordinal-regression
    Number of channels: 3
    Meaning of labels: {'0': '0', '1': '1', '2': '2

In [17]:
from MedVit import MedViT_small as small
model = small()


initialize_weights...


In [18]:
from torch import nn
model.proj_head[0] = nn.Linear(in_features=1024, out_features=n_classes, bias=True)

In [19]:
#Define loss function and optimizer
if task == "multi-task, binary-class":
  criterion = nn.BCEWithLogitsLoss()
else:
  criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9)

In [20]:
criterion

CrossEntropyLoss()

In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [22]:
from torch.nn.functional import softmax
from tqdm import tqdm
#training
for epoch in range(NUM_EPOCHS):
  model = model.to(device)
  model.train()
  for inputs, targets in tqdm(train_loader):
    inputs, targets = inputs.to(device), targets.to(device)
    optimizer.zero_grad()
    outputs = model(inputs.to(torch.float32))
    if task == "multi-label, binary-class":
      targets = targets.to(torch.float32).unsqueeze(1)
    else:
      targets = targets.to(torch.long)
      targets = targets.view(-1)

    predicted_classes = torch.argmax(outputs, dim=1)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()

    # Delete tensors to free u.ep memory
    del inputs, targets, outputs, predicted_classes
    # Empty the cache to clear up some memory
    torch.cuda.empty_cache


100%|██████████| 72/72 [00:33<00:00,  2.13it/s]


In [24]:
from sklearn.metrics import roc_auc_score, accuracy_score

# switch to evaluation mode
def test(split):
  model.eval()
  # Lists to store actual and predicted values
  actuals = []
  probas = []
  predictions = []

  if split == 'val':
        data_loader = train_loader_at_eval
  else:
        data_loader = test_loader

  with torch.no_grad():
      for inputs, targets in tqdm(data_loader):

          inputs, targets = inputs.to(device), targets.to(device)

          if task == 'multi-label, binary-class':
              targets = targets.to(torch.float32).unsqueeze(1)
          else:
              targets = targets.to(torch.long)
              targets = targets.view(-1)

          outputs = model(inputs.to(torch.float32))
          softmax_outputs = softmax(outputs, dim=1)

          _, predicted_classes = torch.max(outputs, 1)

          # Store the actual targets and predicted probabilities
          actuals.extend(targets.cpu().numpy())
          probas.extend(softmax_outputs.detach().cpu().numpy())
          # Probability of positive class
          predictions.extend(predicted_classes.cpu().numpy())
  print('\n')
  auc = roc_auc_score(actuals, probas, multi_class='ovr')
  accuracy = accuracy_score(actuals, predictions)
  print("actuals:",actuals)
  print("predictions:",predictions)
  print('AUC of the model:', auc)
  print('Accuracy of the model:', accuracy)

print('==> Evaluating...')
test('val')
test('test')


==> Evaluating...


100%|██████████| 4/4 [00:02<00:00,  1.42it/s]




actuals: [0, 2, 0, 0, 0, 0, 1, 2, 0, 3, 2, 0, 0, 0, 0, 0, 3, 2, 1, 3, 0, 1, 0, 2, 0, 0, 1, 0, 0, 3, 3, 3, 4, 0, 0, 3, 3, 1, 0, 0, 0, 0, 0, 0, 1, 1, 2, 3, 2, 4, 0, 0, 0, 3, 2, 0, 4, 3, 0, 0, 0, 0, 2, 2, 0, 2, 0, 2, 0, 1, 3, 2, 3, 0, 3, 0, 0, 0, 2, 1, 2, 2, 0, 2, 2, 0, 2, 2, 2, 3, 4, 0, 1, 0, 2, 4, 0, 2, 0, 0, 3, 4, 3, 2, 1, 2, 0, 2, 0, 2, 2, 0, 3, 1, 0, 3, 3, 0, 0, 0]
predictions: [3, 3, 0, 2, 3, 0, 3, 3, 3, 3, 3, 0, 0, 3, 3, 0, 3, 3, 1, 3, 0, 3, 0, 3, 0, 0, 3, 0, 0, 0, 3, 3, 3, 0, 3, 3, 3, 3, 0, 0, 0, 0, 0, 3, 3, 1, 3, 1, 2, 1, 0, 0, 0, 3, 3, 0, 3, 3, 3, 3, 3, 0, 3, 3, 0, 1, 1, 3, 0, 3, 3, 3, 3, 0, 3, 0, 3, 0, 1, 3, 3, 3, 0, 3, 2, 0, 3, 1, 3, 3, 3, 3, 1, 0, 3, 3, 0, 3, 1, 3, 3, 3, 3, 1, 3, 3, 0, 3, 0, 3, 0, 0, 3, 3, 3, 3, 3, 0, 0, 3]
AUC of the model: 0.8153121639554897
Accuracy of the model: 0.49166666666666664


100%|██████████| 14/14 [00:04<00:00,  2.88it/s]



actuals: [2, 1, 0, 0, 0, 0, 2, 2, 0, 3, 2, 2, 0, 3, 0, 0, 0, 2, 4, 0, 0, 0, 0, 0, 2, 0, 0, 3, 0, 3, 0, 0, 3, 1, 0, 0, 0, 3, 1, 0, 1, 0, 0, 0, 3, 3, 1, 1, 1, 0, 0, 3, 3, 0, 1, 1, 2, 0, 0, 1, 0, 0, 4, 0, 2, 2, 0, 1, 0, 3, 0, 1, 0, 2, 3, 1, 4, 3, 0, 0, 2, 0, 0, 4, 3, 0, 0, 1, 3, 0, 2, 1, 0, 2, 3, 4, 2, 1, 0, 0, 0, 3, 0, 0, 0, 4, 0, 0, 3, 0, 2, 2, 0, 2, 0, 0, 2, 1, 3, 2, 3, 2, 0, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 1, 2, 0, 2, 0, 2, 0, 1, 0, 1, 2, 2, 1, 1, 0, 1, 0, 3, 0, 2, 0, 3, 0, 3, 0, 0, 0, 0, 0, 0, 3, 0, 4, 3, 2, 3, 1, 0, 0, 2, 2, 0, 2, 0, 1, 0, 0, 0, 2, 0, 0, 0, 3, 3, 2, 0, 2, 3, 1, 3, 0, 2, 4, 2, 3, 0, 1, 4, 0, 3, 0, 2, 2, 1, 0, 2, 0, 0, 2, 2, 0, 0, 1, 0, 2, 2, 2, 0, 3, 2, 1, 2, 0, 2, 0, 3, 0, 0, 2, 0, 0, 0, 1, 2, 0, 4, 3, 1, 2, 0, 0, 0, 1, 0, 2, 3, 3, 2, 1, 2, 2, 0, 0, 4, 0, 0, 2, 4, 3, 4, 3, 3, 2, 0, 0, 0, 1, 0, 3, 0, 3, 0, 1, 1, 0, 0, 4, 0, 2, 3, 0, 2, 2, 2, 1, 3, 3, 0, 3, 2, 1, 0, 0, 2, 3, 0, 2, 0, 3, 3, 3, 0, 1, 0, 3, 0, 0, 0, 0, 0, 2, 3, 3, 1, 2, 3, 0, 2, 2, 3, 2, 2, 2, 0, 2, 2, 0